**FITS文件的保存与创建**  
@Author: Ray  
@Time: 2022.09.17  
@Cite: https://docs.astropy.org/en/stable/io/fits/index.html#save-file-changes

In [1]:
import numpy as np
from pathlib import Path

from astropy.io import fits

# ^ 示例的输出文件所在路径
PATH_example = Path("/Users/rui/Code/1_Astronote/02_Astropy/example-data") / '3_1_2_fits-save'
PATH_example.mkdir(parents=True, exist_ok=True)
print(f"示例文件存放在 => {PATH_example}")

示例文件存放在 => /Users/rui/Code/1_Astronote/02_Astropy/example-data/3_1_2_fits-save


> FITS文件的数据结构

FITS文件支持多层扩展，成为FITS Cube

* [PrimeryHDU](https://docs.astropy.org/en/stable/io/fits/api/hdus.html?highlight=PrimaryHDU#astropy.io.fits.PrimaryHDU)
    1. 每个FITS文件有且只有一个PrimaryHDU, 第0个扩展必须是PrimaryHDU
    2. PrimaryHDU可以存储image，但不可以存table（因为image数据只是简单的数组，而table有更复杂的数据结构）
* [ImageHDU](https://docs.astropy.org/en/stable/io/fits/api/images.html#astropy.io.fits.ImageHDU)
    1. 图像数据专用的FITS扩展
* [BinTableHDU](https://docs.astropy.org/en/stable/io/fits/api/tables.html#astropy.io.fits.BinTableHDU)
    1. 二进制表格专用的HDU
* [TableHDU](https://docs.astropy.org/en/stable/io/fits/api/tables.html#astropy.io.fits.TableHDU)
    1. ASCII表专用的HDU，比存成二进制更大，一般不用


｜ [BinTableHDU和TableHDU的区别](https://docs.astropy.org/en/stable/io/fits/usage/unfamiliar.html?highlight=TableHDU#creating-an-ascii-table)：
  * FITS标准存储表格时支持2种格式: binary（二进制表格）和ASCII表
  * ASCII表中，由于数据存成人们可读的形式，因此花费更多空间和额外的处理过程，早期的fits会存这个格式
  * 现在推荐使用二进制表格

---
## 创建空的HDU

In [2]:
hdu_primary = fits.PrimaryHDU()
hdu_image = fits.ImageHDU()
print(hdu_primary.data)
print(hdu_image.data)

None
None


In [3]:
hdu_primary.header  # 默认PrimryHDU的header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  

In [4]:
hdu_image.header  # 默认ImageHDU的header

XTENSION= 'IMAGE   '           / Image extension                                
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
PCOUNT  =                    0 / number of parameters                           
GCOUNT  =                    1 / number of groups                               

---
## 创建FITS for Image

In [5]:
# 生成实验用的图像数组
data = np.arange(100.).reshape(10, 10)
data

array([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.],
       [10., 11., 12., 13., 14., 15., 16., 17., 18., 19.],
       [20., 21., 22., 23., 24., 25., 26., 27., 28., 29.],
       [30., 31., 32., 33., 34., 35., 36., 37., 38., 39.],
       [40., 41., 42., 43., 44., 45., 46., 47., 48., 49.],
       [50., 51., 52., 53., 54., 55., 56., 57., 58., 59.],
       [60., 61., 62., 63., 64., 65., 66., 67., 68., 69.],
       [70., 71., 72., 73., 74., 75., 76., 77., 78., 79.],
       [80., 81., 82., 83., 84., 85., 86., 87., 88., 89.],
       [90., 91., 92., 93., 94., 95., 96., 97., 98., 99.]])

### 将图像保存在PrimaryHDU

In [6]:
hdu = fits.PrimaryHDU(data=data)  # 将数据存进HDU
hdul = fits.HDUList([hdu])  # 将此HDU添加到FITS文件的HDUList中
hdul.writeto(PATH_example / 'new1.fits', overwrite=True)  # 将HDUList写入FITS文件

In [7]:
# 检查一下刚刚创建的FITS
hdul_check = fits.open(PATH_example / 'new1.fits')
hdul_check.info()

Filename: /Users/rui/Code/1_Astronote/02_Astropy/example-data/3_1_2_fits-save/new1.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       6   (10, 10)   float64   


### 将图像保存在ImageHDU

In [8]:
hdu_primary = fits.PrimaryHDU()  # FITS文件的第0个扩展必须是PrimaryHDU, 不然存进文件的时候会报错
hdu_image = fits.ImageHDU(data=data, name='test_image')
hdul = fits.HDUList([hdu_primary, hdu_image])
hdul.info()

Filename: (No file associated with this HDUList)
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1  TEST_IMAGE    1 ImageHDU         8   (10, 10)   float64   


---
## 创建FITS for Table  
* Table不能存进PrimaryHDU, 因为其结构的复杂

### 直接将Table存进FITS（快速方法）
* Table的`write()`函数会自动将astropy的Table存进FITS的第1个扩展

In [9]:
from astropy.table import Table
tbl = Table([[1, 2], [4, 5], [7, 8]], names=('a', 'b', 'c'))
tbl

a,b,c
int64,int64,int64
1,4,7
2,5,8


In [10]:
tbl.write(PATH_example / 'new3.fits', overwrite=True)

In [11]:
# 检查一下刚刚创建的FITS
hdul_check = fits.open(PATH_example / 'new3.fits', overwrite=True)
hdul_check.info()

Filename: /Users/rui/Code/1_Astronote/02_Astropy/example-data/3_1_2_fits-save/new3.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1                1 BinTableHDU     14   2R x 3C   [K, K, K]   


### 将一列一列的数据存进FITS（需要知道的方法）

In [12]:
# 将数组存进fits列
c1 = fits.Column(name='a', array=np.array([1, 2]), format='K')
c2 = fits.Column(name='b', array=np.array([4, 5]), format='K')
c3 = fits.Column(name='c', array=np.array([7, 8]), format='K')
c1

name = 'a'; format = 'K'

In [13]:
# 将这些列存进一个hdu
hdu = fits.BinTableHDU.from_columns([c1, c2, c3])
hdu.data

FITS_rec([(1, 4, 7), (2, 5, 8)],
         dtype=(numpy.record, [('a', '<i8'), ('b', '<i8'), ('c', '<i8')]))

In [14]:
# 快捷方法，自动将hdu(Table)写入FITS
hdu.writeto(PATH_example / 'new4.fits', overwrite=True)

In [15]:
# 检查一下刚刚创建的FITS
hdul_check = fits.open(PATH_example / 'new4.fits', overwrite=True)
hdul_check.info()

Filename: /Users/rui/Code/1_Astronote/02_Astropy/example-data/3_1_2_fits-save/new4.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1                1 BinTableHDU     14   2R x 3C   [K, K, K]   


### 写入Table的扩展（标准方法）

In [16]:
# 创建一个Table
tbl = Table([[1, 2], [4, 5], [7, 8]], names=('a', 'b', 'c'))
tbl

a,b,c
int64,int64,int64
1,4,7
2,5,8


In [17]:
# 创建header
header = fits.Header()  # 创建一个空header
header['Author'] = 'Ray'
header

AUTHOR  = 'Ray     '                                                            

使用`fits.TableHDU`
* data不能是Table数据类型，只能是数组

In [18]:
hdu_table = fits.TableHDU(data=tbl.as_array(), header=header, name='table1')
print(repr(hdu_table.header))  # 注意参数xtension
hdu_table.data

XTENSION= 'TABLE   '           / ASCII table extension                          
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                   63 / length of dimension 1                          
NAXIS2  =                    2 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                    3 / number of table fields                         
AUTHOR  = 'Ray     '                                                            
EXTNAME = 'TABLE1  '           / extension name                                 
TTYPE1  = 'a       '                                                            
TFORM1  = 'I21     '                                                            
TBCOL1  =                   

FITS_rec([(1, 4, 7), (2, 5, 8)],
         dtype=(numpy.record, [('a', 'S21'), ('b', 'S21'), ('c', 'S21')]))

使用`fits.BinTableHDU`
* data可以直接是Table

In [19]:
hdu_bintable = fits.BinTableHDU(data=tbl, header=header, name='table2')
print(repr(hdu_bintable.header))  # 注意参数xtension
hdu_bintable.data


XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                   24 / length of dimension 1                          
NAXIS2  =                    2 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                    3 / number of table fields                         
AUTHOR  = 'Ray     '                                                            
EXTNAME = 'TABLE2  '           / extension name                                 
TTYPE1  = 'a       '                                                            
TFORM1  = 'K       '                                                            
TTYPE2  = 'b       '        

FITS_rec([(1, 4, 7), (2, 5, 8)],
         dtype=(numpy.record, [('a', '<i8'), ('b', '<i8'), ('c', '<i8')]))

In [20]:
hdu_primary = fits.PrimaryHDU()  # 创建一个空的PrimaryHDU
hdul = fits.HDUList([hdu_primary, hdu_table, hdu_bintable])  # 将hdu放进hdulist
hdul.writeto(PATH_example / 'new5.fits', overwrite=True)  # 将HDUList存进fits文件中

In [21]:
# 检查一下刚刚创建的FITS
hdul_check = fits.open(PATH_example / 'new5.fits', overwrite=True)
hdul_check.info()

Filename: /Users/rui/Code/1_Astronote/02_Astropy/example-data/3_1_2_fits-save/new5.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1  TABLE1        1 TableHDU        19   2R x 3C   [I21, I21, I21]   
  2  TABLE2        1 BinTableHDU     16   2R x 3C   [K, K, K]   


## 创建HDUList并写入FITS

In [22]:
# 产生收集的数据
image_data = np.arange(100.).reshape(10, 10)
table_data = Table([[1, 2], [4, 5], [7, 8]], names=('a', 'b', 'c'))

In [23]:
header = fits.Header()  # 创建一个空Header
hdu_primary = fits.PrimaryHDU()  # 创建一个主HDU
hdu_image = fits.ImageHDU(data=image_data, header=header, name="my_image")  # 创建图像hdu
hdu_table = fits.BinTableHDU(data=table_data, header=header, name="my_table")  # 创建tableHDU

hdul = fits.HDUList([hdu_primary, hdu_image, hdu_table])  # 创建hdul list

hdul.writeto(PATH_example / 'new6.fits', overwrite=True)  # 将hdul写入fits文件

In [24]:
# 检查一下刚刚创建的FITS
hdul_check = fits.open(PATH_example / 'new6.fits', overwrite=True)
hdul_check.info()

Filename: /Users/rui/Code/1_Astronote/02_Astropy/example-data/3_1_2_fits-save/new6.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1  MY_IMAGE      1 ImageHDU         8   (10, 10)   float64   
  2  MY_TABLE      1 BinTableHDU     15   2R x 3C   [K, K, K]   
